# The Viewer

In [ ]:
import numpy as np
import json
import time
import ipywidgets as widgets
from cad_viewer_widget import CadViewer, AnimationTrack
from sidecar import Sidecar

names = ["hexapod", "hexapod_m", "box", "box1", "boxes", "edges", "faces", "vertices", "pylones", "box-edges", "box-faces", "box-vertices"]
objects = {}
states = {}

for name in names:
    with open(f"examples/{name}.json", "r") as fd:
        objects[name] = json.load(fd)

    with open(f"examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

## Open viewer in cell

In [ ]:
cv1 = CadViewer(
    cad_width=640, 
    height=480, 
    tools=True,
    pinning=True,
)
name = "hexapod"
cv1.add_shapes(
        objects[name],
        states[name],
        ortho=False,
        control="orbit",
        grid=(False, False, False),
)

In [ ]:
cv2 = CadViewer(
    cad_width=640, 
    height=480, 
    tools=True,
    pinning=True
)
name = "faces"
cv2.add_shapes(
    objects[name],
    states[name],
    ortho=False,
    control="orbit",
    grid=(False, False, False),
)

In [ ]:
from ipywidgets.embed import embed_minimal_html
# !!! sc.close()
embed_minimal_html('export.html', views=[cv2.widget], title='Widgets export')

In [ ]:
sc = Sidecar(title="Tests")

with sc:
    cv = CadViewer(
        cad_width=640, 
        height=480, 
        theme="light"
    )

## Observe Javascript events

In [ ]:
cv.js_debug = True

## Adding shapes

In [ ]:
def show(name):
    cv.add_shapes(
        objects[name],
        states[name],
        ortho=True,
        control="trackball",
        ticks=20,
        axes=True,
        axes0=False,
        grid=[True, False, False], 
        transparent=False,
        black_edges=False,
        edge_color="#707070",
        ambient_intensity=0.5,
        direct_intensity=0.3,
        reset_camera=True,
    #    position=[-580.0, 200.0, 750.0],
    #    quaternion=[-0.33, 0.026, 0.91, 0.24],
    #    zoom=0.5,
    #    timeit=True,
    #    animation_loop=False
    )

menu = widgets.Dropdown(
    options=names,
    value=names[0],
    description='Number:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        show(change['new'])

menu.observe(on_change)

show(names[0])
menu

# Camera position control

In [ ]:
p0 = cv.position
if cv.control == "trackball":
    q0 = cv.quaternion
else:
    q0 = None
p0, q0

In [ ]:
cv.position = [-580.0, 200.0, 750.0]

if cv.control == "trackball":
    cv.quaternion = [-0.33, 0.026, 0.91, 0.24]

cv.zoom = 0.5

In [ ]:
cv.position = p0

if cv.control == "trackball":
    cv.quaternion = q0
    
cv.zoom = 1

# Widget interaction

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.edge_color

In [ ]:
cv.edge_color = "#ff0000"

In [ ]:
cv.edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed =1
cv.pan_speed =1
cv.rotate_speed =1

In [ ]:
cv.last_pick

# Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = not cv.clip_intersection

In [ ]:
cv.clip_planes = not cv.clip_planes

In [ ]:
cv.clip_value_0 = 10
cv.clip_value_1 = -50
cv.clip_value_2 = 40

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.select_tree()

# Rotations

In [ ]:
for i in range(10):
    cv.rotate_x(1)
    cv.rotate_y(3)
    cv.rotate_z(5)
    time.sleep(0.1)

In [ ]:
for i in range(10):
    cv.rotate_z(-5)
    cv.rotate_y(-3)
    cv.rotate_x(-1)
    time.sleep(0.1)    

In [ ]:
cv.rotate_up(30)
cv.rotate_left(30)

In [ ]:
cv.rotate_left(-30)
cv.rotate_up(-30)

# Animation

In [ ]:
cv = cv2

In [ ]:
import numpy as np
horizontal_angle = 25

leg_names = {
    "right_back", "right_middle", "right_front", 
    "left_back",  "left_middle", "left_front",
    
}

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1).tolist()
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

leg_group = ("left_front", "right_middle", "left_back")

In [ ]:
tracks = []

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(f"/bottom/{name}", "rz", *horizontal(4, "middle" in name)))

cv.animate(5)
cv.play()

In [ ]:
cv.stop()

In [ ]:
for name in leg_names:
    # move lower leg
    cv.add_track(AnimationTrack(f"/bottom/{name}/lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)))
    
cv.animate(2)
cv.play()

In [ ]:
[a.to_array() for a in cv.tracks]

In [ ]:
cv.clear_tracks()

# No Widget, Javascript only

In [ ]:
import json
from IPython.display import Javascript, HTML
from cad_viewer_widget.utils import serializer
from uuid import uuid4
 
def embed(
    shapes, states, cad_width=800, height=600, tree_width=240, theme="light", tools=True, 
    tracks=None, ortho=True, control="trackball", axes=False, axes0=False, grid=None, ticks=10, normal_len=0, 
    transparent=False, black_edges=False, edge_color="#707070", ambient_intensity=0.9, direct_intensity=0.12, 
    position=None, quaternion=None, zoom=None, 
    reset_camera=True, zoom_speed=1.0, pan_speed=1.0, rotate_speed=1.0):
    
    uid = str(uuid4())
    display(HTML(f"""
        <div id='cad_view_{uid}'></div>
        <script type="text/javascript">
           console.log("XXX");
        </script>
    """))
    if grid is None:
        grid_js = '[false, false, false]'
    else:
        grid_js = json.dumps(grid)
        
    display(Javascript(
    f"""
    function render() {{
        const options = {{
            theme: "{theme}",
            ortho: {json.dumps(ortho)},
            control: "{control}",
            tools: {json.dumps(tools)},
            normalLen: {normal_len},
            cadWidth: {cad_width},
            height: {height},
            treeWidth: {tree_width},
            ticks: {ticks},
            normalLen: {normal_len},
            edgeColor: "{edge_color}",
            ambientIntensity: {ambient_intensity},
            directIntensity: {direct_intensity},
            transparent: {json.dumps(transparent)},
            blackEdges: {json.dumps(black_edges)},
            axes: {json.dumps(axes)},
            grid: {grid_js}, 
            rotateSpeed: {rotate_speed},
            panSpeed: {pan_speed},
            zoomSpeed: {zoom_speed}
        }};
        console.log(options)

        const container = document.getElementById("cad_view_{uid}");
        const display = new CadViewer.Display(container, options);
        display.setAnimationControl(false)
        
        const shapes = {shapes};
        const states = {states};
        const viewer = new CadViewer.Viewer(display, options);
        const tessAndTree = viewer.renderTessellatedShapes(shapes, states);
        viewer.render(
            ...tessAndTree, states,
            {json.dumps(position)},
            {json.dumps(quaternion)},
            {json.dumps(zoom)}
        )
    }}
    console.log("INIT")
    var viewerLib = document.createElement('script');
    viewerLib.type = 'text/javascript';
    viewerLib.src = 'https://unpkg.com/three-cad-viewer@1.0.0/dist/three-cad-viewer.js'

    if(window.CadViewer == null) {{
        console.log("Loading three-cad-viewer ...")
        viewerLib.onload = () => {{
            render()
        }}
        document.head.appendChild(viewerLib);
    }} else {{
       render()
    }}
    """))

In [ ]:
name = "box1"
sh = json.dumps(objects[name], default=serializer)
st = json.dumps(states[name])

embed(sh, st, tools=False, axes=True, transparent=True)

In [ ]:
import ipywidgets as w
w.IntSlider(3,0,10)

In [ ]:
import zlib
from base64 import b64encode

class ShapeEncode(json.JSONEncoder):
    def default(self, obj):
        print(type(obj))
        def compress(obj, dtype):
            cobj = zlib.compress(obj.reshape(-1).astype(dtype).tobytes(order="C"))
            return b64encode(cobj).decode()

        if isinstance(obj, np.ndarray):
            if obj.dtype in (np.float32, np.float64):
                return ("_f32", obj.shape, compress(obj, "float32"))
            elif obj.dtype in (np.int32, np.int64):
                return ("_i32", obj.shape, compress(obj, "int32"))
            else:
                raise Exception("unknown numpy type")
        raise TypeError("Unknown type:", type(obj))


In [ ]:
import numpy as np
name="hexapod"
len(b64encode(zlib.compress(json.dumps(objects[name]).encode("utf-8"))))

In [ ]:
len(json.dumps(objects[name]).encode("utf-8"))

In [ ]:
print(f"<div id='cad_view_{23}' onload='console.log(\"XXX\")'></div>")

In [ ]:
def complex_encoder(z):
    print(z)
    if isinstance(z, complex):
        return (z.real, z.imag)
    else:
        type_name = z.__class__.__name__
        raise TypeError(f"Object of type {type_name} is not serializable")
        
json.dumps(4+6j, default=complex_encoder)

In [ ]:
sc.close()

In [ ]:
import ipywidgets as w
out = w.Output()
out

In [ ]:
from PIL import Image, ImageDraw

def line(output_path):
    image = Image.new("RGB", (400, 400), "lightgray")
    points = [(100, 100), (150, 200), (200, 50), (400, 400)]
    draw = ImageDraw.Draw(image)
    draw.line(points, width=15, fill="green", joint="curve")
    return image

with out:
    display(line("jointed_lines.jpg"))